In [ ]:
# Setup dependencies (as taken from assignment 6)
import os
import math
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models

from torchsummary import summary
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

#Additional Setup to use Tensorboard
!pip install -q tensorflow
%load_ext tensorboard

# Load raw data here, normalise values 

# Create sliding windows here

In [ ]:
#can refer here: 
# https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
# https://curiousily.com/posts/time-series-forecasting-with-lstm-for-daily-coronavirus-cases/
#example 
# def split_data(stock, lookback):
#     data_raw = stock.to_numpy() # convert to numpy array
#     data = []
    
#     # create all possible sequences of length seq_len
#     for index in range(len(data_raw) - lookback): 
#         data.append(data_raw[index: index + lookback])
    
#     data = np.array(data);
#     test_set_size = int(np.round(0.2*data.shape[0]));
#     train_set_size = data.shape[0] - (test_set_size);
    
#     x_train = data[:train_set_size,:-1,:]
#     y_train = data[:train_set_size,-1,:]
    
#     x_test = data[train_set_size:,:-1]
#     y_test = data[train_set_size:,-1,:]
    
#     return [x_train, y_train, x_test, y_test]
# lookback = 20 # choose sequence length
# x_train, y_train, x_test, y_test = split_data(price, lookback)

# x_train = torch.from_numpy(x_train).type(torch.Tensor)
# x_test = torch.from_numpy(x_test).type(torch.Tensor)
# y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
# y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)

# convert it to pytorch datasets

In [ ]:
b_size = 100

# Convert 1D MNIST data to pytorch tensors
tensors_train = torch.Tensor(data['x']), torch.Tensor(data['y']).long()
tensors_test = torch.Tensor(data['x_test']),torch.Tensor(data['y_test']).long()

# Create training set and test set from tensors
train_set = TensorDataset(*tensors_train)
test_set = TensorDataset(*tensors_test)

# Create dataloaders from the training and test set for easier iteration over the data
train_loader = DataLoader(train_set, batch_size=b_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=b_size, shufffle=False)

# Get some data and check for dimensions
input, label = next(iter(train_loader))

# Define LSTM model here

In [ ]:
input_dim = 1 #if input dataset is like [[x1,x2,x3...],y], no. of features in input, if weather, price and time used, then maybe it should be 3
hidden_dim = 10 #no. of neurons in hidden layer
num_layers = 2 # no of hidden layers 
output_dim = 1 #if we predict only 1 value
num_epochs = 100

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_() #hidden layer output
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_() 
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        # Index hidden state of last time step
        out = self.fc(out[:, -1, :]) 
        return out


In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
print(model)

# training loop

In [ ]:

# Create a writer to write to Tensorboard
writer = SummaryWriter()

# Some hyperparams
epochs = 80


lstm = []
for t in range(num_epochs):
    # Train one epoch
    
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    writer.add_scalars(' Loss',
                        {'Train': loss },
                        epoch)
    
print('\nFinished.')
writer.flush()
writer.close()

In [ ]:
N=7#we use on week worth of data
T=24# we use one day of previous data for training
D= #len each training sample

In [ ]:
model.eval()
# refer here: https://www.kaggle.com/taronzakaryan/predicting-stock-price-using-lstm-model-pytorch
#unnormlaise values: inverse transform
# example code
# y_test_pred = model(x_test)

# # invert predictions
# y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
# y_train = scaler.inverse_transform(y_train.detach().numpy())
# y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
# y_test = scaler.inverse_transform(y_test.detach().numpy())

# # calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
# print('Train Score: %.2f RMSE' % (trainScore))
# testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
# print('Test Score: %.2f RMSE' % (testScore))



In [ ]:
# run this cell once and wait for it to time out
# run this cell a second time and you should see the board

%tensorboard --logdir runs/ --host localhost